# Typed Objects

In addition to the interfaces offered by `ObjectAPIClient` and `DownloadedObject`, which provides access to Geoscience Objects in a way that is agnostic to the specific object type, the `evo-objects` package also provides a set of "typed objects" that represent specific object types.

As usual, we need to first authenticate, which we can do using the `ServiceManagerWidget` from the `evo-notebooks` package

In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="your-client-id", cache_location="./notebook-data"
).login()

## Creating a regular 3D grid object

To create a new 'regular-3d-grid' object, we use the `Regular3DGridData` class to define all of the properties of the grid, then pass that to the `Regular3DGrid.create` class method.

The `cell_data` and `vertex_data` properties are dataframes that define the attributes of the cells and vertices respectively. Each column in the dataframe represents a different attribute.

Note the order of the data is important, and must match the order of the cells and vertices in the grid. The cells are ordered such that the x-axis varies the fastest, then y, then z. So for a grid of size (10, 10, 5), the first 10 rows of the dataframe correspond to the cells at (0, 0, 0) to (9, 0, 0), the next 10 rows correspond to (0, 1, 0) to (9, 1, 0), and so on.

In [ ]:
import numpy as np
import pandas as pd

from evo.objects.typed import Point3, Regular3DGrid, Regular3DGridData, Rotation, Size3d, Size3i

grid = Regular3DGridData(
    name="Test Grid",
    origin=Point3(0, 0, 0),
    size=Size3i(10, 10, 5),
    cell_size=Size3d(2.5, 5, 5),
    cell_data=pd.DataFrame(
        {
            "value": np.random.rand(10 * 10 * 5),
            "cat": pd.Categorical.from_codes(np.random.choice(range(4), size=10 * 10 * 5), ["a", "b", "c", "d"]),
        }
    ),
    vertex_data=pd.DataFrame(
        {
            "elevation": np.random.rand(11 * 11 * 6),
        }
    ),
    rotation=Rotation(90, 0, 0),
)
created_grid = await Regular3DGrid.create(manager, grid)
print(f"Created regular-3d-grid object: {created_grid.metadata.url}")

## Downloading a regular 3D grid object

To download an existing 'regular-3d-grid' object, and inspect it, you can use the `Regular3DGrid.from_reference` class method. This method takes the context and the URL of the object to download.

For the example, we will download the object we just created.

In [ ]:
grid = await Regular3DGrid.from_reference(manager, created_grid.metadata.url)

print(f"Downloaded regular-3d-grid object with name: {grid.name}")
print(f"Origin: {grid.origin}")
print(f"Size: {grid.size}")
print(f"Cell Size: {grid.cell_size}")
print(f"Rotation: {grid.rotation}")
print(f"Bounding Box: {grid.bounding_box}")

To inspect the data, `grid.cells.as_dataframe()`, and `grid.vertices.as_dataframe()` can be used to get the cell and vertex data as pandas DataFrames.

In [ ]:
await grid.cells.as_dataframe()

In [ ]:
await grid.vertices.as_dataframe()

## Updating a regular 3D grid object

To update an existing 'regular-3d-grid' object, we can modify the properties of the `Regular3DGrid` instance, and then call the `update` method to submit the changes to the Geoscience Object service. This will create a new version of the object with the updated properties.

Updating the cell or vertex data can be done by calling the `set_dataframe` method on the `cells` or `vertices` attributes respectively. This will upload the new data to the Geoscience Object service, but the object will remain the same version until the `update` method is called.

In [ ]:
print(f"Version before update: {grid.metadata.version_id}")

grid.name = "Updated Test Grid"

await grid.cells.set_dataframe(
    pd.DataFrame(
        {
            "value": np.ones(10 * 10 * 5),
        }
    )
)

# Update the object to create a new version with the modified properties
await grid.update()
print(f"Updated version: {grid.metadata.version_id}")